In [16]:
import pymysql
import pandas as pd
from pyspark.sql import SparkSession

In [17]:
def mysql_connect():
    return pymysql.connect(host='localhost', port=3306, user='user', password='pass123', db='uol')
sql = 'SELECT * FROM registration'

In [18]:
# Get pdf from pandas
pdf = pd.read_sql(sql, con=mysql_connect())

In [19]:
# Create data frame from pdf with spark
spark = SparkSession.builder.appName('singleUserService').getOrCreate()
df = spark.createDataFrame(pdf)
df.show()

+---------------+------------------+---------------------+-----------------+----------------------+-----------------------+-------------------------+----------------------------+-------------------+----------------------+---------------+
|registration_id|registration_email|registration_password|registration_name|registration_cellphone|registration_user_agent|registration_name_browser|registration_version_browser|registration_system|registration_gpu_model|registration_ip|
+---------------+------------------+---------------------+-----------------+----------------------+-----------------------+-------------------------+----------------------------+-------------------+----------------------+---------------+
|              1|  teste5@teste.com|                teste|            teste|           12999999999|            user_agent2|              nameBrowser|              versionBrowser|             system|              gpuModel|             ip|
+---------------+------------------+------------